In [1]:
import pandas as pd
import numpy as np
from os import listdir, path

In [2]:
p = 'datasets'

def resumo(arquivo):
    metadado = pd.read_csv(arquivo, encoding='latin_1', sep=';', decimal=',', nrows=8, header=None)
    local = f'{metadado.iloc[0, 1]} -- {metadado.iloc[1, 1]} -- {metadado.iloc[3,1]} -- {metadado.iloc[2,1]}'
    
    dados = pd.read_csv(arquivo, skiprows=8, encoding='latin_1', sep=';', decimal=',').replace(-9999, np.nan)
    return dados.min().rename(local), dados.max().rename(local), (dados.isnull().sum(axis=0)/len(dados)*100).rename(local), dados.apply(lambda x: x.groupby(x.notna().cumsum()).cumcount().max()).rename(local)

def resumo_anual(ano):
    tabelas = {
        'min': [], #minimo de cada coluna
        'max': [], #maximo de cada coluna
        'porcentagem': [], #porcentagem de nulos em cada coluna
        'sequencia': []  #maior sequencia de nulos
    }
    
    d = f'{p}/{ano}/'
    if path.isdir(f'{d}{ano}'): d = f'{d}{ano}/'
        
    for file in listdir(d):
        minimo, maximo, porcentagem, sequencia = resumo(d+file)
        tabelas['min'].append(minimo)
        tabelas['max'].append(maximo)
        tabelas['porcentagem'].append(porcentagem)
        tabelas['sequencia'].append(sequencia)
    
    for k, v in tabelas.items():
        df = pd.DataFrame(v)
        df.columns = [f'{k.upper()} - {c}' for c in df.columns]
        tabelas[k] = df
        
    colunas = [v.columns for k, v in tabelas.items()]
    colunas = [j for i in zip(*colunas) for j in i]
    return pd.concat(tabelas.values(), axis=1).reindex(columns=colunas).sort_index()#

In [3]:
todos = {}
for ano in range(2000, 2022):
    print(f'Procesando ano {ano}...', end='')
    df = resumo_anual(ano)
    todos[ano] = df
    #df.to_csv(f'{ano}.csv', sep=';', decimal=',', encoding='latin_1')
    print('OK')

Procesando ano 2000...OK
Procesando ano 2001...OK
Procesando ano 2002...OK
Procesando ano 2003...OK
Procesando ano 2004...OK
Procesando ano 2005...OK
Procesando ano 2006...OK
Procesando ano 2007...OK
Procesando ano 2008...OK
Procesando ano 2009...OK
Procesando ano 2010...OK
Procesando ano 2011...OK
Procesando ano 2012...OK
Procesando ano 2013...OK
Procesando ano 2014...OK
Procesando ano 2015...OK
Procesando ano 2016...

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/2016/2016/INMET_NE_MA_A225_IMPERATRIZ_01-01-2016_A_31-12-2016.CSV'

In [21]:
filtrado = [(k, v[v.index.str.startswith(('S -- SC', 'SE -- SP'))]) for k, v in todos.items()]
pd.DataFrame([v['PORCENTAGEM - PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].rename(k) for k, v in filtrado if not v.empty]).T.sort_index().iloc[:, 14:].applymap(lambda x: 'x' if x < 1/13*100 else '').to_csv('meus_porcentagem.csv', sep=';', decimal=',', encoding='latin_1')

<NA>